In [2]:
import torch
training_device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
training_device
import numpy as np

In [3]:
#!pip install evaluate datasets transformers accelerate==0.26.0

In [4]:
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict

df = pd.read_csv("data/acc_data.csv")
df.head()

,recordId,gpt4o_judge_score,nova_judge_score,llama3_judge_score,majority_value,agreement_percentage,writing_id,task_id,level_title,activity_instructions,student_submission
0,CALL0000322,3.0,4.0,3.0,3.0,66.666667,08e77bf2-2a0d-4098-b5ec-40f593ca6336,de7a1159-0c06-49bf-a273-7bdd195d6a32,11-Upper Intermediate,You are applying for a position as a manager a...,I am writing to apply for the manager position...
1,CALL0002886,0.0,0.0,0.0,0.0,100.000000,f6a2b5e6-ceb8-4575-ac74-ada1ab9f8cc7,7ae47a96-722c-4b2b-ad07-3716f2aef85f,8-Intermediate,You're going to write an email about some futu...,You're going to write an email about some futu...
2,CALL0001805,5.0,5.0,5.0,5.0,100.000000,f8823fe1-ca4b-43dc-9685-85009a105efa,cfb893f1-bee1-4894-99f3-43af284493a5,3-Beginner,Write a paragraph about a job your friend has ...,My friend João worked at TechNova Soluções fro...
3,CALL0000693,5.0,4.0,5.0,5.0,66.666667,2ddd84ba-aebc-4808-b270-ca3bc19cb15a,03849587-d03b-45ed-8534-c34536867a8a,12-Upper Intermediate,Write a letter to apologize to someone for som...,"Dear Alex,\n\nI want to sincerely apologize fo..."
4,CALL0000291,5.0,5.0,5.0,5.0,100.000000,57ac94ab-2084-4a23-a6bf-61d71cd1900a,23c3d59a-61d5-470f-90d1-1217b0006929,10-Upper Intermediate,"An old friend has emailed you, describing what...",Subject: So Good to Hear From You! \n\nStan! ...


In [5]:
df['text'] = (
    "Prompt Level: " + df['level_title'].astype(str) +
    " [SEP] Prompt: " + df['activity_instructions'] +
    " [SEP] Response: " + df['student_submission']
)


In [6]:
df.head()

,recordId,gpt4o_judge_score,nova_judge_score,llama3_judge_score,majority_value,agreement_percentage,writing_id,task_id,level_title,activity_instructions,student_submission,text
0,CALL0000322,3.0,4.0,3.0,3.0,66.666667,08e77bf2-2a0d-4098-b5ec-40f593ca6336,de7a1159-0c06-49bf-a273-7bdd195d6a32,11-Upper Intermediate,You are applying for a position as a manager a...,I am writing to apply for the manager position...,Prompt Level: 11-Upper Intermediate [SEP] Prom...
1,CALL0002886,0.0,0.0,0.0,0.0,100.000000,f6a2b5e6-ceb8-4575-ac74-ada1ab9f8cc7,7ae47a96-722c-4b2b-ad07-3716f2aef85f,8-Intermediate,You're going to write an email about some futu...,You're going to write an email about some futu...,Prompt Level: 8-Intermediate [SEP] Prompt: You...
2,CALL0001805,5.0,5.0,5.0,5.0,100.000000,f8823fe1-ca4b-43dc-9685-85009a105efa,cfb893f1-bee1-4894-99f3-43af284493a5,3-Beginner,Write a paragraph about a job your friend has ...,My friend João worked at TechNova Soluções fro...,Prompt Level: 3-Beginner [SEP] Prompt: Write a...
3,CALL0000693,5.0,4.0,5.0,5.0,66.666667,2ddd84ba-aebc-4808-b270-ca3bc19cb15a,03849587-d03b-45ed-8534-c34536867a8a,12-Upper Intermediate,Write a letter to apologize to someone for som...,"Dear Alex,\n\nI want to sincerely apologize fo...",Prompt Level: 12-Upper Intermediate [SEP] Prom...
4,CALL0000291,5.0,5.0,5.0,5.0,100.000000,57ac94ab-2084-4a23-a6bf-61d71cd1900a,23c3d59a-61d5-470f-90d1-1217b0006929,10-Upper Intermediate,"An old friend has emailed you, describing what...",Subject: So Good to Hear From You! \n\nStan! ...,Prompt Level: 10-Upper Intermediate [SEP] Prom...


In [7]:
df = df[["text", "majority_value"]]
df = df.rename(columns={'majority_value': 'label'})
df.head()

,text,label
0,Prompt Level: 11-Upper Intermediate [SEP] Prom...,3.0
1,Prompt Level: 8-Intermediate [SEP] Prompt: You...,0.0
2,Prompt Level: 3-Beginner [SEP] Prompt: Write a...,5.0
3,Prompt Level: 12-Upper Intermediate [SEP] Prom...,5.0
4,Prompt Level: 10-Upper Intermediate [SEP] Prom...,5.0


In [8]:
# Apply the mapping to the 'labels' column
#df['label'] = df['label'].map(label_mapping)
df.dropna(subset=['label'], inplace=True)
df.reset_index(drop=True, inplace=True)

df.head()

,text,label
0,Prompt Level: 11-Upper Intermediate [SEP] Prom...,3.0
1,Prompt Level: 8-Intermediate [SEP] Prompt: You...,0.0
2,Prompt Level: 3-Beginner [SEP] Prompt: Write a...,5.0
3,Prompt Level: 12-Upper Intermediate [SEP] Prom...,5.0
4,Prompt Level: 10-Upper Intermediate [SEP] Prom...,5.0


In [9]:
df["label"].value_counts()

label
5.0    930
2.0    556
4.0    466
3.0    351
0.0    340
1.0    118
Name: count, dtype: int64

In [10]:
ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['text', 'label'],
    num_rows: 2761
})

In [11]:
from datasets import ClassLabel, Value, Sequence
new_features = ds.features.copy()
new_features["label"] = ClassLabel(names=[0, 1, 2, 3, 4])
ds = ds.cast(new_features)

# Step 1: Initial train/test split with stratification
train_test_ds = ds.train_test_split(test_size=0.20, seed=20)

# Step 2: Split the test set into half test, half validation
test_valid_split = train_test_ds['test'].train_test_split(test_size=0.5, seed=20)

# Step 3: Combine everything into a single DatasetDict
ds = DatasetDict({
    'train': train_test_ds['train'],
    'test': test_valid_split['train'],    # This becomes the test set
    'validation': test_valid_split['test']  # This becomes the validation set
})
ds

Casting the dataset:   0%|          | 0/2761 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 2208
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 276
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 277
    })
})

In [12]:
# Verify label distribution
from collections import Counter

print("Train label counts:", Counter(ds['train']['label']))
print("Test label counts:", Counter(ds['test']['label']))
print("Validation label counts:", Counter(ds['validation']['label']))

Train label counts: Counter({5: 732, 2: 447, 4: 381, 3: 278, 0: 274, 1: 96})
Test label counts: Counter({5: 98, 2: 57, 4: 44, 3: 35, 0: 34, 1: 8})
Validation label counts: Counter({5: 100, 2: 52, 4: 41, 3: 38, 0: 32, 1: 14})


In [13]:
import json

# Fonction utilitaire pour sauvegarder un split en JSONL
def save_split_to_jsonl(dataset_split, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for record in dataset_split:
            f.write(json.dumps(record, ensure_ascii=False) + '\n')

# Sauvegarde des trois splits
save_split_to_jsonl(ds['train'], 'data/train.jsonl')
save_split_to_jsonl(ds['test'], 'data/test.jsonl')
save_split_to_jsonl(ds['validation'], 'data/validation.jsonl')

In [ ]:
# save data